In [189]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

with open('latex_homework.txt', 'r', encoding='utf-8') as f:
    text1 = f.read()

with open('tiny_shakespeare.txt', 'r', encoding='utf-8') as f:
    text2 = f.read()

with open('jokes.txt', 'r', encoding='utf-8') as f:
    text3 = f.read()

text = text1


In [191]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l]) 


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [192]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [193]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 

        wei = q @ k.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v 
        return out


In [194]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [195]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [201]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

       
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
       
        for _ in range(max_new_tokens):
            
            idx_cond = idx[:, -block_size:]
            
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :] # becomes (B, C)
            
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [202]:
batch_size = 16 
block_size = 32 
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'mps' if torch.mps.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0
# ------------


model = BigramLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    
    xb, yb = get_batch('train')

    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.822118 M parameters
step 0: train loss 4.7523, val loss 4.7393
step 100: train loss 2.7491, val loss 2.7557
step 200: train loss 2.4684, val loss 2.5337
step 300: train loss 2.2355, val loss 2.3307
step 400: train loss 2.0533, val loss 2.1565
step 500: train loss 1.9196, val loss 2.0507
step 600: train loss 1.8321, val loss 1.9678
step 700: train loss 1.7390, val loss 1.8852
step 800: train loss 1.7042, val loss 1.8648
step 900: train loss 1.6553, val loss 1.8343
step 1000: train loss 1.6031, val loss 1.7973
step 1100: train loss 1.5602, val loss 1.7788
step 1200: train loss 1.5207, val loss 1.7352
step 1300: train loss 1.5073, val loss 1.7368
step 1400: train loss 1.4765, val loss 1.6890
step 1500: train loss 1.4661, val loss 1.6960
step 1600: train loss 1.4178, val loss 1.6879
step 1700: train loss 1.4136, val loss 1.6885
step 1800: train loss 1.3838, val loss 1.6481
step 1900: train loss 1.3686, val loss 1.6194
step 2000: train loss 1.3640, val loss 1.6282
step 2100: train loss 1.

In [238]:
learning_rate *= 0.01
max_iters = 1000

for iter in range(max_iters):

    
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    
    xb, yb = get_batch('train')

    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.8784, val loss 1.5889
step 100: train loss 0.8808, val loss 1.5877
step 200: train loss 0.8723, val loss 1.5849
step 300: train loss 0.8744, val loss 1.5809
step 400: train loss 0.8757, val loss 1.5834
step 500: train loss 0.8656, val loss 1.6163
step 600: train loss 0.8653, val loss 1.5659
step 700: train loss 0.8653, val loss 1.5658
step 800: train loss 0.8547, val loss 1.5717
step 900: train loss 0.8493, val loss 1.5732
step 999: train loss 0.8589, val loss 1.5761


In [240]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

	n{theorem}
    
\end{tikzpicture}

\item Let $\left(X, d_X\right)$ is a point $\cos \right)$ and $\limrc = x^{n+1}a_n
\end{gather*}
\begin{gather*}
     s_m[\alpha_n = -\frac{k^2}{N'} \sumni (0,1) [ut-v(x')] = 4cx'\theta(x+-\infty)
\end{gather*}
Note indly equal\beta u' = -\frac{g(x)}{\lambda(x)P(x)} = Q(x)
\end{align*}
No's and \(\|f_n'' - x' - \|_{\lambda x} \right| \\
&< \frac{1}{\sqrt{2\pi}}\frac{\pi}{2}\left[\cos\frac{(x)-f(x+x)^2+\Delta)}{II}
\end{gather*}
Where $\psi(n-1) = \frac{1}{e^L} 


First, and its what $E(r,r) = \zeta)$, we cally closurary negrespa


$$
{\rightarrow L^{-1}} \quad u_n(x) = \sin{\int_{E_\alpha} g'(x)} - e^{-i\omega}
$$

$$
b_n=a^{n} = a_0 \delta (x,t) + b\delta\sin{x\in L^2} = 1
$$

$$
t^3r = \frac{1}{2\pi} \sin k(x'-L)
$$

$$
\lambda = \int dx' p(x) f(x) \delta
$$

$$
u^{n-1}u'(0) = \lim_{n\to \infty} \frac{1}{n}\int_\Omega^{-i\alpha}ke={e^{-\alpha,ex} = 0}
$$

$$
\beta u' = -\frac{g(x)}{\lambda(x)P(x)} = Q(x)
$$

Let $\left(X, d_X \right)$ is a point $\left( \cos \right)$ and $\lim rc = x^{n+1}a_n$

Where $\psi(n-1) = \frac{1}{e^L}$